## 여러 데이터로 trained model로 가격 예측

In [1]:
import pandas as pd
from xgboost import XGBRegressor
import numpy as np
import pickle

# XGBoost 모델 파일 불러오기
# 저장된 모델 로드
with open('remove_camera_XGBRegressor_best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 새로운 데이터셋 불러오기
new_seller_df = pd.read_excel('C:/Python_practice/DL_project/final_project/no_Yongsan_yes_Doksan/DA/dataset/new_seller_df.xlsx')

In [2]:
from phone_CrawlingDataPreprocessor import DataPreprocessor
prepro = DataPreprocessor("C:/Python_practice/DL_project/final_project/no_Yongsan_yes_Doksan/DA/dataset/phone_input_data.csv")
origin_df = prepro.execute() 
origin_df

,price,quality,favoritecount,viewcount,product_name,screen_size,cost,capacity,ram,weight,Max_video_playtime,Max_audio_playtime,battery_capacity,hours,used_year
8137,450000,2.0,0.0,0.000000,아이폰 12 Mini,0.291289,0.429971,256,0.171856,0.000000,0.183447,0.574961,0.199267,0.000000,0.965326
9748,970000,2.0,0.0,0.000000,갤럭시 Z 폴드4,0.990334,0.833867,256,0.828144,0.907189,0.753284,0.769301,0.873439,0.000000,0.832952
21572,530000,3.0,0.0,0.052656,아이폰 12,0.533182,0.334492,128,0.171856,0.262451,0.261801,0.714791,0.431242,0.000000,0.965326
19929,950000,4.0,0.0,0.000338,갤럭시 S 23 플러스,0.694572,0.544222,256,0.585928,0.509143,1.000000,0.851457,0.938740,0.052677,0.000000
22773,560000,3.0,0.0,0.038424,아이폰 13 Mini,0.291289,0.281714,128,0.171856,0.068249,0.399137,0.625758,0.275807,0.083491,0.916461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11023,750000,4.0,0.0,0.010130,갤럭시 Z 폴드2,1.000000,0.969077,256,0.828144,1.000000,0.666667,0.845074,0.895688,0.993387,0.965326
9744,750000,2.0,0.0,0.029286,갤럭시 Z 폴드2,1.000000,0.969077,256,0.828144,1.000000,0.666667,0.845074,0.895688,1.000000,0.965326
12807,1050000,2.0,0.0,0.052630,갤럭시 Z 폴드2,1.000000,0.969077,256,0.828144,1.000000,0.666667,0.845074,0.895688,1.000000,0.965326
12796,1200000,2.0,0.0,0.017214,갤럭시 Z 폴드3,0.988180,0.833867,256,0.828144,0.947059,0.666667,0.698134,0.873439,1.000000,0.916461


In [4]:
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9331 entries, 8137 to 11048
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               9331 non-null   int64  
 1   quality             9331 non-null   float64
 2   favoritecount       9331 non-null   float64
 3   viewcount           9331 non-null   float64
 4   product_name        9331 non-null   object 
 5   screen_size         9331 non-null   float64
 6   cost                9331 non-null   float64
 7   capacity            9331 non-null   int32  
 8   ram                 9331 non-null   float64
 9   weight              9331 non-null   float64
 10  Max_video_playtime  9331 non-null   float64
 11  Max_audio_playtime  9331 non-null   float64
 12  battery_capacity    9331 non-null   float64
 13  hours               9331 non-null   float64
 14  used_year           9331 non-null   float64
dtypes: float64(12), int32(1), int64(1), object(1)
memor

In [3]:
new_seller_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_name        5 non-null      object 
 1   capacity            5 non-null      int64  
 2   quality             5 non-null      object 
 3   favoritecount       0 non-null      float64
 4   viewcount           0 non-null      float64
 5   release_year        0 non-null      float64
 6   screen_size         0 non-null      float64
 7   weight              0 non-null      float64
 8   Max_video_playtime  0 non-null      float64
 9   Max_audio_playtime  0 non-null      float64
 10  battery_capacity    0 non-null      float64
 11  ram                 0 non-null      float64
 12  cost                0 non-null      float64
 13  time                0 non-null      float64
 14  price               5 non-null      int64  
dtypes: float64(11), int64(2), object(2)
memory usage: 728.0+ byte

In [5]:
# 판매자가 제시할 텍스트 데이터 (input data) + 사진을 통해 등급 데이터가 들어온다는 가정
# 제품명(product_name), 사양(capacity), 등급(quality)
# new_seller_df 파일

# time은 기본 "3분 전"으로 입력
new_seller_df['time'] = new_seller_df['time'].apply(lambda x : '3분 전')

# seller의 데이터에서 사양 매칭
# 사양 연결
def map_phone_info(df):    
    # capacity 열 타입 변환(object -> int)
    df['capacity'] = df['capacity'].astype(int)
    
    # 4. product_name, capacity 기준으로 사양들 맵핑
    phone_info = pd.read_csv('phone_info_v2.csv')
    phone_info = phone_info.drop_duplicates(subset=['product', 'capacity'])
    
    # product, capacity 열을 인덱스로 설정
    phone_info = phone_info.set_index(['product', 'capacity']) 

    # prepro_df의 각 열을 phone_info 데이터프레임을 사용하여 업데이트
    for col in ['release_year', 'screen_size', 'weight', 'Max_video_playtime',
                'Max_audio_playtime', 'battery_capacity', 'ram', 'cost']:
        
        # 'product_name'과 'capacity' 열을 zip으로 묶어서 매핑
        df[col] = list(map(lambda x: phone_info.loc[(x[0], x[1]), col] 
                                        if (x[0], x[1]) in phone_info.index else np.nan, 
                                        zip(df['product_name'], df['capacity'])))
    return df

# 시간 통일화
def assign_h_time(df):
    '''
        time 열을 시간 단위로 통일화하여 hours 열에 할당
        실제 실행될 때 크롤링 일자가 모두 동일하다는 가정 하에 진행한 코드입니다.
    '''
    df['hours'] = np.nan
    df['hours'] = df['time'].str.extract('(\d+)\s*시간', expand=False).astype(float)
    df.loc[df['time'].str.contains('분'), 'hours'] = df['time'].str.extract('(\d+)\s*분', expand=False).astype(float) / 60
    df.loc[df['time'].str.contains('일'), 'hours'] = df['time'].str.extract('(\d+)\s*일', expand=False).astype(float) * 24
    df.loc[df['time'].str.contains('달'), 'hours'] = df['time'].str.extract('(\d+)\s*달', expand=False).astype(float) * 720
    df.loc[df['time'].str.contains('년'), 'hours'] = df['time'].str.extract('(\d+)\s*년', expand=False).astype(float) * 8640
    df.drop(['time'], axis=1, inplace=True)

    return df

# 출시 년도 -> 년식으로 변경
def convert_as_used_year(df):
    '''
    2023년 대비 년식을 구하는 함수
    '''
    df['used_year'] = df['release_year'].apply(lambda x: 2023 - x)
    
    # release_year 열 삭제
    df.drop(['release_year'], axis=1, inplace=True)

    return df

# 등급 변환
def extract_and_map_quality(df):
    # 등급 점수화
    quality_mapping = {
        '미개봉': 5,
        'S급': 4,
        'A급': 3,
        'B급': 2,
        'C급': 1,
    }
    df['quality'] = df['quality'].map(quality_mapping)
    return df


new_seller_df = map_phone_info(new_seller_df)
new_seller_df = assign_h_time(new_seller_df)
new_seller_df = convert_as_used_year(new_seller_df)
new_seller_df = extract_and_map_quality(new_seller_df)

In [6]:
# 조회수와 좋아요는 중고사이트의 평균으로 계산해서 데이터에 추가
# favoritecount, viewcount

def calculate_avg_fav_and_view(origin_df, new_seller_df):
    # 전처리 실행(가중치 진행) 후 favoritecount와 viewcount의 평균을 계산하여 avg_info_df 생성
    avg_info = origin_df.groupby(['product_name', 'capacity'])[['favoritecount', 'viewcount']].mean().reset_index()
    avg_info.columns = ['product_name', 'capacity', 'favoritecount_avg', 'viewcount_avg']
    
    # 딕셔너리 생성
    fav_avg_dict = pd.Series(avg_info.favoritecount_avg.values, index=[avg_info.product_name, avg_info.capacity]).to_dict()
    view_avg_dict = pd.Series(avg_info.viewcount_avg.values, index=[avg_info.product_name, avg_info.capacity]).to_dict()

    # seller_df (판매자 데이터)에 평균 대입
    new_seller_df['favoritecount'] = new_seller_df.set_index(['product_name', 'capacity']).index.map(fav_avg_dict)
    new_seller_df['viewcount'] = new_seller_df.set_index(['product_name', 'capacity']).index.map(view_avg_dict)

    return new_seller_df


# 함수 호출하여 평균값 계산 및 데이터 할당
new_seller_df = calculate_avg_fav_and_view(origin_df, new_seller_df)
new_seller_df

,product_name,capacity,quality,favoritecount,viewcount,screen_size,weight,Max_video_playtime,Max_audio_playtime,battery_capacity,ram,cost,price,hours,used_year
0,갤럭시 Z 폴드3,256,2,0.002630,0.033067,19.19,0.271,18,63,"4,400",12,"1,998,700",850000,0.05,2
1,갤럭시 Z 폴드3,256,4,0.002630,0.033067,19.19,0.271,18,63,"4,400",12,"1,998,700",580000,0.05,2
2,갤럭시 Z 플립4,256,2,0.000593,0.009725,17.03,0.187,19,56,"3,700",8,"1,353,000",630000,0.05,1
3,갤럭시 Z 플립4,256,1,0.000593,0.009725,17.03,0.187,19,56,"3,700",8,"1,353,000",470000,0.05,1
4,갤럭시 Z 플립4,256,3,0.000593,0.009725,17.03,0.187,19,56,"3,700",8,"1,353,000",560000,0.05,1


In [7]:
model_df = new_seller_df.copy()
model_df.drop(['product_name'], axis=1, inplace=True)

# 타입 변환, 로그변환, 정규화 : favoritecount, viewcount 제외한 모든 열

def change_data_type(df):
    df['cost'] = df['cost'].str.replace(',', '').astype(int)
    df['ram'] = df['ram'].astype(int)
    df['weight'] = df['weight'].astype(float)
    df['Max_video_playtime'] = df['Max_video_playtime'].astype(int)
    df['Max_audio_playtime'] = df['Max_audio_playtime'].astype(int)
    df['battery_capacity'] = df['battery_capacity'].str.replace(',', '').astype(int)

    return df

# 로그 변환 전 준비
# 0인 데이터에 0.001 추가하기 (최대한 데이터 손상을 막기 위해)
model_df['used_year'] = model_df['used_year'].apply(lambda x: x + 0.001)

# 로그 변환
def log_transform(dataframe, columns):
    '''
    데이터프레임과 컬럼을 인자로 받아서
    로그 변환이 적용된 데이터프레임을 반환하는 함수
    # price 종속변수 제외
    '''
    for column in columns:
        if column not in ['price', 'favoritecount', 'viewcount']: 
            filtered = dataframe[column][dataframe[column] > 0] # filter out negative or zero values
            dataframe[column] = np.log(filtered) # apply log transformation
    return dataframe

# 정규화
def get_minmax_scaler(df, exclude_columns=['price', 'favoritecount', 'viewcount']): 
    '''
    price(종속변수) 열을 제외한 스케일링 함수
    '''
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaled_df = df.copy()

    for column in df.columns:
        if column not in exclude_columns: 
            scaled_values = scaler.fit_transform(scaled_df[[column]])  # Scale the values
            scaled_df[column] = scaled_values  # Assign the scaled values back to the dataframe
    return scaled_df

model_df = change_data_type(model_df)
model_df = log_transform(model_df, model_df.columns)
model_df = get_minmax_scaler(model_df)
model_df

,capacity,quality,favoritecount,viewcount,screen_size,weight,Max_video_playtime,Max_audio_playtime,battery_capacity,ram,cost,price,hours,used_year
0,0.0,0.500000,0.002630,0.033067,1.0,1.0,0.0,1.0,1.0,1.0,1.0,850000,0.0,1.0
1,0.0,1.000000,0.002630,0.033067,1.0,1.0,0.0,1.0,1.0,1.0,1.0,580000,0.0,1.0
2,0.0,0.500000,0.000593,0.009725,0.0,0.0,1.0,0.0,0.0,0.0,0.0,630000,0.0,0.0
3,0.0,0.000000,0.000593,0.009725,0.0,0.0,1.0,0.0,0.0,0.0,0.0,470000,0.0,0.0
4,0.0,0.792481,0.000593,0.009725,0.0,0.0,1.0,0.0,0.0,0.0,0.0,560000,0.0,0.0


In [8]:
# price 열 제외
model_df.drop(['price'], axis=1, inplace=True)

In [9]:
# 모델에 학습될 때 feature(독립변수)의 순서와 판매자의 데이터의 feature의 순서가 같아야 함
model_columns_list = ['quality', 'favoritecount', 'viewcount', 'screen_size', 'cost', 'capacity', 'ram', 'weight', 'Max_video_playtime', 'Max_audio_playtime', 'battery_capacity', 'hours', 'used_year']
model_df = model_df[model_columns_list]

In [10]:
# 예측 수행
y_pred = loaded_model.predict(model_df)

for i, prediction in enumerate(y_pred):
    print(f"Data {i+1}'s prediction: {prediction}")

Data 1's prediction: 464136.34375
Data 2's prediction: 828505.9375
Data 3's prediction: 576458.875
Data 4's prediction: 599594.625
Data 5's prediction: 604984.0625


In [14]:
# 기종별 가격 분포
mean_price_df = origin_df.copy()

pd.set_option('display.float_format', '{:.2f}'.format)

# 기종별, 용량별, 품질별 가격 분포 (평균, 최소, 최대)
price_stats_per_product_capacity_quality = mean_price_df.groupby(['product_name', 'capacity', 'quality'])['price'].agg(['mean', 'min', 'max'])

price_stats_per_product_capacity_quality_df = price_stats_per_product_capacity_quality.reset_index()
price_stats_per_product_capacity_quality_df


,product_name,capacity,quality,mean,min,max
0,갤럭시 S 20,128,2.00,641249.75,435000,999999
1,갤럭시 S 20,128,3.00,466666.67,450000,500000
2,갤럭시 S 20,128,4.00,425333.33,410000,446000
3,갤럭시 S 20,128,5.00,1600000.00,750000,2450000
4,갤럭시 S 20 울트라,256,2.00,419884.62,403000,500000
...,...,...,...,...,...,...
287,아이폰 SE(3세대),128,2.00,495625.00,420000,580000
288,아이폰 SE(3세대),128,3.00,410000.00,405000,415000
289,아이폰 SE(3세대),128,4.00,465571.43,440000,550000
290,아이폰 SE(3세대),128,5.00,485769.23,410000,550000


In [17]:
price_stats_per_product_capacity_quality_df.loc[(price_stats_per_product_capacity_quality_df['product_name'] == '갤럭시 Z 폴드3') | 
                                                (price_stats_per_product_capacity_quality_df['product_name'] == '갤럭시 Z 플립4')]


,product_name,capacity,quality,mean,min,max
69,갤럭시 Z 폴드3,256,2.00,730443.18,420000,1800000
70,갤럭시 Z 폴드3,256,3.00,687168.90,410000,1100000
71,갤럭시 Z 폴드3,256,4.00,751722.88,430000,1150000
72,갤럭시 Z 폴드3,256,5.00,1087257.69,850000,1998700
73,갤럭시 Z 폴드3,512,2.00,782242.09,499999,1200000
74,갤럭시 Z 폴드3,512,3.00,794021.74,567000,1150000
75,갤럭시 Z 폴드3,512,4.00,818153.54,480000,1950000
76,갤럭시 Z 폴드3,512,5.00,1248000.00,600000,2000000
96,갤럭시 Z 플립4,256,2.00,682159.09,530000,2200000
97,갤럭시 Z 플립4,256,3.00,619615.38,530000,930000


갤럭시 Z 폴드3/256/2/850000 - Data 1's prediction: 464136.34375<br>
갤럭시 Z 폴드3/256/4/580000 - Data 2's prediction: 828505.9375<br>
갤럭시 Z 플립4/256/2/630000  - Data 3's prediction: 576458.875<br>
갤럭시 Z 플립4/256/1/470000 - Data 4's prediction: 599594.625<br>
갤럭시 Z 플립4/256/3/560000  - Data 5's prediction: 604984.0625<br>

## 판매자 입력 데이터 py 파일 오류 점검

In [61]:
import pandas as pd
import numpy as np
import re
from phone_CrawlingDataPreprocessor import DataPreprocessor
from phone_CrawlingDataPreprocessor import modeling_processing
from phone_CrawlingDataPreprocessor import remove_y
from xgboost import XGBRegressor
import numpy as np
import pickle

class seller_data_processor:
    '''
    # 판매자가 입력할 텍스트 데이터 : 제품명(product_name), 사양(capacity)
    # 사진을 통해 등급 데이터가 텍스트로 들어온다는 가정 : 등급(quality)
    '''
    def __init__(self, product_name, capacity, quality):
        self.product_name = product_name
        self.capacity = capacity
        self.quality = quality

        self.min_price = None
        self.max_price = None
        self.predicted_price = None

    def basic_adding(self):
        # 데이터프레임 형성
        self.new_seller_df = pd.DataFrame(columns = ['product_name','quality', 'favoritecount', 'viewcount', 'release_year','screen_size', 'cost', 'capacity', 'ram', 'weight', 'Max_video_playtime', 'Max_audio_playtime', 'battery_capacity', 'time'])

        # 데이터프레임에 행 추가
        self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)

        # 각 열에 값 할당
        self.new_seller_df['product_name'] = self.product_name
        self.new_seller_df['capacity'] = self.capacity
        self.new_seller_df['quality'] = self.quality

        # time은 기본 "3분 전"으로 입력
        self.new_seller_df['time'] = '3분 전'


    # 사양 연결
    def map_phone_info(self):    
        # capacity 열 타입 변환(object -> int)
        self.new_seller_df['capacity'] = self.new_seller_df['capacity'].astype(int)
        
        # product_name, capacity 기준으로 사양들 맵핑
        phone_info = pd.read_csv('phone_info_v2.csv')
        phone_info = phone_info.drop_duplicates(subset=['product', 'capacity'])
        
        # product, capacity 열을 인덱스로 설정
        phone_info = phone_info.set_index(['product', 'capacity']) 

        # new_seller_df의 각 열을 phone_info 데이터프레임을 사용하여 업데이트
        for col in ['release_year', 'screen_size', 'weight', 'Max_video_playtime',
                    'Max_audio_playtime', 'battery_capacity', 'ram', 'cost']:
            
            # 'product_name'과 'capacity' 열을 zip으로 묶어서 매핑
            self.new_seller_df[col] = list(map(lambda x: phone_info.loc[(x[0], x[1]), col] 
                                            if (x[0], x[1]) in phone_info.index else np.nan, 
                                            zip(self.new_seller_df['product_name'], self.new_seller_df['capacity'])))

    # 시간 통일화
    def assign_h_time(self):
        '''
         time 열을 시간 단위로 통일화하여 hours 열에 할당
         실제 실행될 때 크롤링 일자가 모두 동일하다는 가정 하에 진행한 코드입니다.
        '''
        self.new_seller_df['hours'] = np.nan
        self.new_seller_df['hours'] = self.new_seller_df['time'].str.extract('(\d+)\s*시간', expand=False).astype(float)
        self.new_seller_df.loc[self.new_seller_df['time'].str.contains('분'), 'hours'] = self.new_seller_df['time'].str.extract('(\d+)\s*분', expand=False).astype(float) / 60
        self.new_seller_df.loc[self.new_seller_df['time'].str.contains('일'), 'hours'] = self.new_seller_df['time'].str.extract('(\d+)\s*일', expand=False).astype(float) * 24
        self.new_seller_df.loc[self.new_seller_df['time'].str.contains('달'), 'hours'] = self.new_seller_df['time'].str.extract('(\d+)\s*달', expand=False).astype(float) * 720
        self.new_seller_df.loc[self.new_seller_df['time'].str.contains('년'), 'hours'] = self.new_seller_df['time'].str.extract('(\d+)\s*년', expand=False).astype(float) * 8640
        self.new_seller_df.drop(['time'], axis=1, inplace=True)

    # 출시 년도 -> 년식으로 변경
    def convert_as_used_year(self):
        '''
        2023년 대비 년식을 구하는 함수
        '''
        self.new_seller_df['used_year'] = self.new_seller_df['release_year'].apply(lambda x: 2023 - x)
        
        # release_year 열 삭제
        self.new_seller_df.drop(['release_year'], axis=1, inplace=True)

    # 등급 변환
    def extract_and_map_quality(self):
        # 등급 점수화
        quality_mapping = {
            '미개봉': 5,
            'S급': 4,
            'A급': 3,
            'B급': 2,
            'C급': 1,
        }
        self.new_seller_df['quality'] = self.new_seller_df['quality'].map(quality_mapping)

    # favoritecount와 viewcount의 평균 업데이트 + min, max price 정의
    def calculate_avg_fav_and_view(self):
        '''
        모든 판매자의 조회수와 좋아요 -> 중고사이트의 평균으로 계산해서 데이터에 추가

        # 매일 업데이트되는 크롤링 데이터에서 전처리 진행 + 스케일링 진행한 데이터에서
        # favoritecount와 viewcount의 평균을 계산하여 avg_info_df 생성
        '''
        # 전처리만 진행한 df 불러오기 
        prepro = DataPreprocessor("C:/Python_practice/DL_project/final_project/no_Yongsan_yes_Doksan/DA/dataset/phone_input_data.csv")
        origin_df = prepro.execute()
        merged_df = origin_df[['product_name', 'capacity']]

        # 기종별, 용량별, 품질별 가격 분포 (평균, 최소, 최대) # 이 내용이 db에 저장되면 코드 수정
        pd.set_option('display.float_format', '{:.2f}'.format)
        price_stats = origin_df.groupby(['product_name', 'capacity', 'quality'])['price'].agg(['min', 'max'])

        price_stats = price_stats.reset_index()
        quality_value = self.new_seller_df['quality'].values[0]

        self.min_price = price_stats.loc[(price_stats['product_name'] == self.product_name) & 
                                         (price_stats['capacity'] == self.capacity) &
                                         (price_stats['quality'] == quality_value), 'min'].values[0]
        self.max_price = price_stats.loc[(price_stats['product_name'] == self.product_name) & 
                                         (price_stats['capacity'] == self.capacity) &
                                         (price_stats['quality'] == quality_value), 'max'].values[0]
        
        # 스케일링 진행한 df 불러오기
        scaled = modeling_processing(origin_df)
        scaled_origin_df = scaled.execute()
        
        # favoritecount와 viewcount의 평균 (스케일링 전의 df(pre_merged_df)기준으로 groupby 진행 후 scaled_origin_df의 찜, 조회수의 평균 필요)
        # 두 데이터프레임 병합
        merged_df['favoritecount'] = scaled_origin_df['favoritecount']
        merged_df['viewcount'] = scaled_origin_df['viewcount']

        # # 병합된 데이터프레임에서 그룹바이를 수행하고 평균을 계산
        avg_info = merged_df.groupby(['product_name', 'capacity'])[['favoritecount', 'viewcount']].mean().reset_index()
        avg_info.columns = ['product_name', 'capacity', 'favoritecount_avg', 'viewcount_avg']

        # 딕셔너리 생성
        fav_avg_dict = pd.Series(avg_info.favoritecount_avg.values, index=[avg_info.product_name, avg_info.capacity]).to_dict()
        view_avg_dict = pd.Series(avg_info.viewcount_avg.values, index=[avg_info.product_name, avg_info.capacity]).to_dict()

        # seller_df (판매자 데이터)에 평균 대입
        self.new_seller_df['favoritecount'] = self.new_seller_df.set_index(['product_name', 'capacity']).index.map(fav_avg_dict)
        self.new_seller_df['viewcount'] = self.new_seller_df.set_index(['product_name', 'capacity']).index.map(view_avg_dict)

    # 타입 변환
    def change_data_type(self):
        self.new_seller_df['cost'] = self.new_seller_df['cost'].str.replace(',', '').astype(int)
        self.new_seller_df['ram'] = self.new_seller_df['ram'].astype(int)
        self.new_seller_df['weight'] = self.new_seller_df['weight'].astype(float)
        self.new_seller_df['Max_video_playtime'] = self.new_seller_df['Max_video_playtime'].astype(int)
        self.new_seller_df['Max_audio_playtime'] = self.new_seller_df['Max_audio_playtime'].astype(int)
        self.new_seller_df['battery_capacity'] = self.new_seller_df['battery_capacity'].str.replace(',', '').astype(int)

    def modeling_Preprocessor(self):
        # product_name 열 삭제
        self.new_seller_df.drop(['product_name'], axis=1, inplace=True)

        # 로그 변환 전 준비 : used_year 열에 0.001 더하기
        # 0인 데이터에 0.001 추가하기 (최대한 데이터 손상을 막기 위해) 
        self.new_seller_df['used_year'] = self.new_seller_df['used_year'].apply(lambda x: x + 0.001)

        # 로그 변환
        def log_transform(dataframe, columns):
            '''
            데이터프레임과 컬럼을 인자로 받아서
            로그 변환이 적용된 데이터프레임을 반환하는 함수
            # price 종속변수 제외
            '''
            for column in columns:
                if column not in ['favoritecount', 'viewcount']: 
                    filtered = dataframe[column][dataframe[column] > 0] # filter out negative or zero values
                    dataframe[column] = np.log(filtered) # apply log transformation
            return dataframe

        self.new_seller_df = log_transform(self.new_seller_df, self.new_seller_df.columns)

        # 정규화
        def get_minmax_scaler(scaled_df, exclude_columns=['favoritecount', 'viewcount']): 
            '''
            단일행일 때는 정규화 불가능 
            => 모델 학습할 때 Train된 minmaxscaler 불러오기
            '''
            import joblib
            # MinMaxScaler 객체 불러오기
            loaded_scaler = joblib.load('scaler.pkl')

            for column in scaled_df.columns:
                if column not in exclude_columns: 
                    # 해당 열의 스케일러를 가져옴
                    scaler = loaded_scaler[column]
                    # nan값 제거 시 
                    if len(scaled_df[[column]].dropna()) > 0:
                        scaled_values = scaler.transform(scaled_df[[column]])  # Scale the values
                        scaled_df[column] = scaled_values  # Assign the scaled values back to the dataframe

            return scaled_df

        self.new_seller_df = get_minmax_scaler(self.new_seller_df)
    
    def load_no_scaling_price_prdict_model(self):
        # XGBoost 모델 파일 불러오기
        with open('remove_camera_XGBRegressor_best_model.pkl', 'rb') as f:
            loaded_model = pickle.load(f)

        # 예측 수행
        self.predicted_price = loaded_model.predict(self.new_seller_df)
        
        # NumPy 배열을 스칼라 값으로 변환
        self.predicted_price = self.predicted_price.item()  

        # 소수점 이하를 버리고 십원(두 번째 자리)까지의 값을 얻음
        formatted_price = "{:.0f}".format(self.predicted_price)
        formatted_price = formatted_price[:-2] + "00"

        self.predicted_price = formatted_price

    def generate_result_string(self):
        if np.isnan(self.min_price) | np.isnan(self.max_price):
            result_string = f"판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 {self.quality}등급입니다.\n"
            result_string += f"{self.product_name} {self.capacity}기가 {self.quality}등급은 오늘 기준으로 판매상품이 아직 없는 것으로 확인되었습니다.\n"
            result_string += f"이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 {self.predicted_price}원입니다."        
        else :
            result_string = f"판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 {self.quality}등급입니다.\n"
            result_string += f"{self.product_name} {self.capacity}기가 {self.quality}등급은 오늘 기준으로 최소 판매가 {self.min_price}원, 최대 판매가 {self.max_price}원입니다.\n"
            result_string += f"이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 {self.predicted_price}원입니다."
        
        return result_string

    def execute(self):
        self.basic_adding()
        self.map_phone_info()
        self.assign_h_time()
        self.convert_as_used_year()
        self.extract_and_map_quality()
        self.calculate_avg_fav_and_view()
        self.change_data_type()
        self.modeling_Preprocessor()
        self.load_no_scaling_price_prdict_model()
        result = self.generate_result_string()
        
        return result

In [62]:
data1 = seller_data_processor('갤럭시 Z 폴드3', 256, 'B급')
data1_result = data1.execute()
data1_result

C:\Users\dohee\AppData\Local\Temp\ipykernel_6620\446585549.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
C:\Users\dohee\AppData\Local\Temp\ipykernel_6620\446585549.py:30: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
C:\Users\dohee\AppData\Local\Temp\ipykernel_6620\446585549.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['favoritecount'] = scaled_origin_df['favoritecount']
C:\Users\d

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 B급등급입니다.\n갤럭시 Z 폴드3 256기가 B급등급은 오늘 기준으로 최소 판매가 420000원, 최대 판매가 1800000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 630000원입니다.'

## 판매자 입력 데이터 py 파일 실행

In [2]:
# 갤럭시 Z 폴드3/256/2/850000 
# remove_camera_xgb : 630000
from Seller_data_Preprocessor import seller_data_processor
data1 = seller_data_processor('갤럭시 Z 폴드3', 256, 'B급')
data1_result = data1.execute()
data1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 B급등급입니다.\n갤럭시 Z 폴드3 256기가 B급등급은 오늘 기준으로 최소 판매가 420000원, 최대 판매가 1800000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 630000원입니다.'

In [3]:
# 갤럭시 Z 폴드3/256/4/580000
# 652000
data2 = seller_data_processor('갤럭시 Z 폴드3', 256, 'S급')
data2_result = data2.execute()
data2_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 S급등급입니다.\n갤럭시 Z 폴드3 256기가 S급등급은 오늘 기준으로 최소 판매가 430000원, 최대 판매가 1150000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 652000원입니다.'

In [4]:
# 갤럭시 Z 플립4/256/2/630000
data3 = seller_data_processor('갤럭시 Z 플립4', 256, 'B급')
data3_result = data3.execute()
data3_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 B급등급입니다.\n갤럭시 Z 플립4 256기가 B급등급은 오늘 기준으로 최소 판매가 530000원, 최대 판매가 2200000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 648700원입니다.'

In [1]:
# 갤럭시 Z 플립4/256/1/470000
from Seller_data_Preprocessor import seller_data_processor
data4 = seller_data_processor('갤럭시 Z 플립4', 256, 'C급')
data4_result = data4.execute()
data4_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 C급입니다.\n갤럭시 Z 플립4 256기가 C급 등급은 오늘 기준으로 판매상품이 아직 없는 것으로 확인되었습니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 648700원입니다.'

In [2]:
# 갤럭시 Z 플립4/256/3/560000
# 641200
data5 = seller_data_processor('갤럭시 Z 플립4', 256, 'A급')
data5_result = data5.execute()
data5_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 A급입니다.\n갤럭시 Z 플립4 256기가 A급 등급은 오늘 기준으로 최소 판매가 530000원, 최대 판매가 930000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 641200원입니다.'

In [4]:
# 아이폰 14 Pro/512/2/
data6 = seller_data_processor('아이폰 14 Pro', 512, 'S급')
data6_result = data6.execute()
data6_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:30: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 S급입니다.\n아이폰 14 Pro 512기가 S급 등급은 오늘 기준으로 최소 판매가 1150000원, 최대 판매가 1800000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 1435000원입니다.'

평균 가격과 비교하기

In [2]:
import numpy as np
import pandas as pd
from phone_CrawlingDataPreprocessor import DataPreprocessor
# 스케일링 전 : DataPreprocessor, 스케일링 시도하려면 phone_CrawlingDataPreprocessor의 modeling_processing 실행해야 함
prepro = DataPreprocessor("C:/Python_practice/DL_project/final_project/no_Yongsan_yes_Doksan/DA/dataset/phone_input_data.csv")
origin_df = prepro.execute() 
origin_df

# 기종별 가격 분포
mean_price_df = origin_df.copy()

pd.set_option('display.float_format', '{:.2f}'.format)

# 기종별, 용량별, 품질별 가격 분포 (평균, 최소, 최대)
price_stats = mean_price_df.groupby(['product_name', 'capacity', 'quality'])['price'].agg(['mean', 'min', 'max'])

price_stats_df = price_stats.reset_index()

In [3]:
price_stats_df.loc[(price_stats_df['product_name'] == '아이폰 14 Pro') & 
                   (price_stats_df['capacity'] == 512)]

,product_name,capacity,quality,mean,min,max
261,아이폰 14 Pro,512,2.00,1532727.27,1330000,1900000
262,아이폰 14 Pro,512,4.00,1452437.50,1150000,1800000
263,아이폰 14 Pro,512,5.00,1720391.30,1600000,1950000


## seller_data_processor 코드 수정 후 시도
6번째 xgb 모델로 진행 (camera 열 제거 + price도 스케일링한 모델)

In [1]:
# 갤럭시 Z 폴드3/256/2/850000 
# data1 : remove_camera_xgb : 630000
# another1 : 1288408.8
from Seller_data_Preprocessor import seller_data_processor
another1 = seller_data_processor('갤럭시 Z 폴드3', 256, 'B급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 B급입니다.\n갤럭시 Z 폴드3 256기가 B급 등급은 오늘 기준으로 최소 판매가 420000원, 최대 판매가 1800000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 [1288408.8]원입니다.'

In [2]:
# 갤럭시 Z 플립4/256/3/560000
# remove_camera_xgb : 641200
# 6번째 xgb : 1289266.8
another2 = seller_data_processor('갤럭시 Z 플립4', 256, 'A급')
another2_result = another2.execute()
another2_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 A급입니다.\n갤럭시 Z 플립4 256기가 A급 등급은 오늘 기준으로 최소 판매가 530000원, 최대 판매가 930000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 [1289266.8]원입니다.'

In [3]:
# 갤럭시 Z 폴드3/256/4/580000
# 652000
# 1295539.8
another3 = seller_data_processor('갤럭시 Z 폴드3', 256, 'S급')
another3_result = another3.execute()
another3_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 S급입니다.\n갤럭시 Z 폴드3 256기가 S급 등급은 오늘 기준으로 최소 판매가 430000원, 최대 판매가 1150000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 [1295539.8]원입니다.'

In [5]:
# 갤럭시 Z 플립4/256/2/630000
# 648700
# 1292418.8
another4 = seller_data_processor('갤럭시 Z 플립4', 256, 'B급')
another4_result = another4.execute()
another4_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 B급입니다.\n갤럭시 Z 플립4 256기가 B급 등급은 오늘 기준으로 최소 판매가 530000원, 최대 판매가 2200000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 [1292418.8]원입니다.'

In [6]:
# 갤럭시 Z 플립4/256/1/470000
# 648700
# 1292418.8
from Seller_data_Preprocessor import seller_data_processor
another5 = seller_data_processor('갤럭시 Z 플립4', 256, 'C급')
another5_result = another5.execute()
another5_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 C급입니다.\n갤럭시 Z 플립4 256기가 C급 등급은 오늘 기준으로 판매상품이 아직 없는 것으로 확인되었습니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 [1292418.8]원입니다.'

In [7]:
# 아이폰 14 Pro/512/2/
# 1435000
# 1547870.1
another6 = seller_data_processor('아이폰 14 Pro', 512, 'S급')
another6_result = another6.execute()
another6_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 S급입니다.\n아이폰 14 Pro 512기가 S급 등급은 오늘 기준으로 최소 판매가 1150000원, 최대 판매가 1800000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 [1547870.1]원입니다.'

price 열도 함께 스케일링 한 이후 역변환 결과값이 너무 높거나 최대 판매가보다 높은 가격도 존재<br> 
=> price를 스케일링하지 않은 모델로 진행

## 기존 모델로 가격 예측에 대한 추가 확인

In [1]:
# 아이폰 SE(3세대)/128/S급/350000
# 가격 예측 : 468000
from Seller_data_Preprocessor import seller_data_processor
another1 = seller_data_processor('아이폰 SE(3세대)', 128, 'S급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 S급입니다.\n아이폰 SE(3세대) 128기가 S급 등급은 오늘 기준으로 최소 판매가 440000원, 최대 판매가 550000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 468000원입니다.'

In [2]:
# 아이폰 14/256/S급/949000
# 942900
another1 = seller_data_processor('아이폰 14', 256, 'S급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 S급입니다.\n아이폰 14 256기가 S급 등급은 오늘 기준으로 최소 판매가 850000원, 최대 판매가 1400000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 942900원입니다.'

In [3]:
# 아이폰 13 Pro/256/A급/990000
# 917900
another1 = seller_data_processor('아이폰 13 Pro', 256, 'A급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 A급입니다.\n아이폰 13 Pro 256기가 A급 등급은 오늘 기준으로 최소 판매가 600000원, 최대 판매가 1142680원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 917900원입니다.'

In [4]:
# 아이폰 13 Pro Max/128/A급/960000
# 996700
another1 = seller_data_processor('아이폰 13 Pro Max', 128, 'A급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 A급입니다.\n아이폰 13 Pro Max 128기가 A급 등급은 오늘 기준으로 최소 판매가 980000원, 최대 판매가 1200000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 996700원입니다.'

In [5]:
# 아이폰 14 Pro Max/256/S급/1500000
# 1414800
another1 = seller_data_processor('아이폰 14 Pro Max', 256, 'S급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 S급입니다.\n아이폰 14 Pro Max 256기가 S급 등급은 오늘 기준으로 최소 판매가 650000원, 최대 판매가 1750000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 1414800원입니다.'

In [6]:
# 아이폰 13 Mini/256/B급/600000
# 636700
another1 = seller_data_processor('아이폰 13 Mini', 256, 'B급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 B급입니다.\n아이폰 13 Mini 256기가 B급 등급은 오늘 기준으로 최소 판매가 420000원, 최대 판매가 999999원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 636700원입니다.'

In [7]:
# 아이폰 14 Plus/128/미개봉/1080000
# 985500
another1 = seller_data_processor('아이폰 14 Plus', 128, '미개봉')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 미개봉입니다.\n아이폰 14 Plus 128기가 미개봉 등급은 오늘 기준으로 최소 판매가 463000원, 최대 판매가 1990000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 985500원입니다.'

In [8]:
# 아이폰 14 Plus/256/S급/1150000
# 998300
another1 = seller_data_processor('아이폰 14 Plus', 256, 'S급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 S급입니다.\n아이폰 14 Plus 256기가 S급 등급은 오늘 기준으로 최소 판매가 999000원, 최대 판매가 1000000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 998300원입니다.'

In [9]:
# 아이폰 14 Plus/512/S급/1420000
# 1216100
another1 = seller_data_processor('아이폰 14 Plus', 512, 'S급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 S급입니다.\n아이폰 14 Plus 512기가 S급 등급은 오늘 기준으로 최소 판매가 1100000원, 최대 판매가 1100000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 1216100원입니다.'

In [10]:
# 갤럭시 S 22 울트라/256/A급/750000
# 708900
another1 = seller_data_processor('갤럭시 S 22 울트라', 256, 'A급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 A급입니다.\n갤럭시 S 22 울트라 256기가 A급 등급은 오늘 기준으로 최소 판매가 480000원, 최대 판매가 1050000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 708900원입니다.'

In [11]:
# 갤럭시 S 22 울트라/512/A급/680000
# 769300
another1 = seller_data_processor('갤럭시 S 22 울트라', 512, 'A급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 A급입니다.\n갤럭시 S 22 울트라 512기가 A급 등급은 오늘 기준으로 최소 판매가 640000원, 최대 판매가 950000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 769300원입니다.'

In [12]:
# 갤럭시노트 20/256/A급/270000
# 535400
another1 = seller_data_processor('갤럭시노트 20', 256, 'A급')
another1_result = another1.execute()
another1_result

c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.new_seller_df = self.new_seller_df.append({}, ignore_index=True)
c:\Python_practice\DL_project\final_project\no_Yongsan_yes_Doksan\DA\Price_predict_model_smartphone\Seller_data_Preprocessor.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

'판매자님이 전달해주신 사진을 확인한 결과, 제품의 등급은 A급입니다.\n갤럭시노트 20 256기가 A급 등급은 오늘 기준으로 최소 판매가 509000원, 최대 판매가 509000원입니다.\n이를 바탕으로 중고사이트의 판매가에 대한 AI 가격 예측 결과는 535400원입니다.'